In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('agri_energy_merged.csv')

In [4]:
df.isna().sum()

Unnamed: 0.1                          0
Unnamed: 0                            0
state                                 0
district                              0
year                                  0
month                                 0
solar_mw_district                 23892
wind_mw                           50028
bioenergy_mw                       4356
solar_pumps_installed             10824
estimated_co2_reduction_tonnes    23892
dtype: int64

In [5]:
# Group by state & district and count nulls column-wise
null_summary = (
    df
    .groupby(['state', 'district'], as_index=False)
    .agg(lambda x: x.isna().sum())
)

# Keep only rows where at least one null exists
null_summary_filtered = null_summary[
    null_summary.drop(columns=['state', 'district']).sum(axis=1) > 0
]

null_summary_filtered.loc[1:20]

,state,district,Unnamed: 0.1,Unnamed: 0,year,month,solar_mw_district,wind_mw,bioenergy_mw,solar_pumps_installed,estimated_co2_reduction_tonnes
3,andhra pradesh,adilabad,0,0,0,0,0,0,0,132,0
4,andhra pradesh,anantapur,0,0,0,0,0,0,0,132,0
5,andhra pradesh,chittoor,0,0,0,0,0,0,0,132,0
6,andhra pradesh,east godavari,0,0,0,0,0,0,0,132,0
7,andhra pradesh,guntur,0,0,0,0,0,0,0,132,0
8,andhra pradesh,hyderabad,0,0,0,0,0,0,0,132,0
9,andhra pradesh,karimnagar,0,0,0,0,0,0,0,132,0
10,andhra pradesh,khammam,0,0,0,0,0,0,0,132,0
11,andhra pradesh,krishna,0,0,0,0,0,0,0,132,0
12,andhra pradesh,kurnool,0,0,0,0,0,0,0,132,0


In [6]:
ap_pumps_2019 = {
    'Vizianagaram': 4225,
    'East Godavari': 3256,
    'Krishna': 2584,
    'Visakhapatnam': 2516,
    'Srikakulam': 1697,
    'West Godavari': 1357
}


In [7]:
rows = []

for district, total_pumps in ap_pumps_2019.items():
    monthly_add = total_pumps / n_months

    cumulative = 0
    for date in monthly_range_phase1:
        cumulative += monthly_add
        rows.append({
            'state': 'Andhra Pradesh',
            'district': district,
            'date': date,
            'year': date.year,
            'month': date.month,
            'solar_pumps_installed': round(cumulative),
        })

phase1_df = pd.DataFrame(rows)
phase1_df.head()


NameError: name 'n_months' is not defined

In [ ]:
PUMP_KW = 3.7

phase1_df['solar_mw_district'] = (
    phase1_df['solar_pumps_installed'] * PUMP_KW / 1000
)


In [ ]:
rows = []

for district in ap_pumps_2019.keys():
    cumulative_mw = phase1_df[
        phase1_df['district'] == district
    ]['solar_mw_district'].max()

    for date in monthly_range_phase2:
        cumulative_mw += monthly_mw_add

        rows.append({
            'state': 'Andhra Pradesh',
            'district': district,
            'date': date,
            'year': date.year,
            'month': date.month,
            'solar_pumps_installed': ap_pumps_2019[district],  # fixed after 2019
            'solar_mw_district': round(cumulative_mw, 2)
        })

phase2_df = pd.DataFrame(rows)


In [ ]:
ap_monthly_df = pd.concat([phase1_df, phase2_df], ignore_index=True)
ap_monthly_df.sort_values(['district']).head(15)


In [ ]:
ap_monthly_df['wind_mw'] = 0.0
ap_monthly_df['bioenergy_mw'] = 0.0


In [ ]:
EMISSION_FACTOR = 0.82  # kg CO2 / kWh

ap_monthly_df['estimated_co2_reduction_tonnes'] = (
    ap_monthly_df['solar_mw_district'] *
    1000 *  # MW → kW
    30 *  # avg days per month
    4.2 * # avg kWh/kWp/day (AP average)
    EMISSION_FACTOR / 1000
).round(2)


In [ ]:
df = pd.concat([df, ap_monthly_df], ignore_index=True)


In [ ]:
df.shape

In [ ]:
# Group by state & district and count nulls column-wise
null_summary = (
    df
    .groupby(['state', 'district'], as_index=False)
    .agg(lambda x: x.isna().sum())
)

# Keep only rows where at least one null exists
null_summary_filtered = null_summary[
    null_summary.drop(columns=['state', 'district']).sum(axis=1) > 0
]

null_summary_filtered.loc[0:20]

In [ ]:
df.columns

In [ ]:
df= df.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

In [ ]:
df = df.drop(['date'], axis=1)

In [ ]:
df.columns

In [ ]:
df.isna().sum()

In [ ]:
# Group by state & district and count nulls column-wise
null_summary = (
    df
    .groupby(['state', 'district'], as_index=False)
    .agg(lambda x: x.isna().sum())
)

# Keep only rows where at least one null exists
null_summary_filtered = null_summary[
    null_summary.drop(columns=['state', 'district']).sum(axis=1) > 0
]

null_summary_filtered.loc[20:40]

In [ ]:
ap_pump_targets = {
    'east godavari': 3256,
    'krishna': 2584,
    'vizianagaram': 4225,
    'visakhapatnam': 2516,
    'srikakulam': 1697,
    'west godavari': 1357
}


In [ ]:
pump_growth_ratio = {
    2015: 0.05,
    2016: 0.20,
    2017: 0.45,
    2018: 0.75,
    2019: 1.00,
    2020: 1.00,
    2021: 1.00,
    2022: 1.00,
    2023: 1.00,
    2024: 1.00,
    2025: 1.00
}


In [8]:
def get_target_pumps(row):
    d = row['district'].lower()
    y = row['year']

    # if district data available
    if d in ap_pump_targets:
        final_pumps = ap_pump_targets[d]
    else:
        final_pumps = 132   # fallback average from your table

    ratio = pump_growth_ratio.get(y, 1.0)
    return round(final_pumps * ratio)


In [9]:
mask_ap = df['state'].str.lower().eq('andhra pradesh')

df.loc[mask_ap, 'solar_pumps_installed'] = (
    df.loc[mask_ap].apply(get_target_pumps, axis=1)
)
mask_ap.sum()

NameError: name 'ap_pump_targets' is not defined

In [ ]:
PUMP_KW = 3.7

df.loc[mask_ap, 'solar_mw_district'] = (
    df.loc[mask_ap, 'solar_pumps_installed'] * PUMP_KW / 1000
).round(3)


In [ ]:
feeder_mw_add = {
    2020: 10,
    2021: 20,
    2022: 40,
    2023: 80,
    2024: 120,
    2025: 150
}


In [ ]:
ap_district_count = df[mask_ap]['district'].nunique()

def add_feeder_mw(row):
    y = row['year']
    add = feeder_mw_add.get(y, 0)
    return add / ap_district_count


In [ ]:
df.loc[mask_ap, 'solar_mw_district'] += (
    df.loc[mask_ap].apply(add_feeder_mw, axis=1)
)


In [10]:
EMISSION_FACTOR = 0.82
AVG_KWH_PER_KWP = 4.2

df.loc[mask_ap, 'estimated_co2_reduction_tonnes'] = (
    df.loc[mask_ap, 'solar_mw_district'] *
    1000 * 30 * AVG_KWH_PER_KWP *
    EMISSION_FACTOR / 1000
).round(2)


In [11]:
df.isna().sum()

Unnamed: 0.1                          0
Unnamed: 0                            0
state                                 0
district                              0
year                                  0
month                                 0
solar_mw_district                 23892
wind_mw                           50028
bioenergy_mw                       4356
solar_pumps_installed             10824
estimated_co2_reduction_tonnes    23892
dtype: int64

In [12]:
# 1. Solar pumps
df.loc[mask_ap, 'solar_pumps_installed'] = df.loc[mask_ap].apply(get_target_pumps, axis=1)

# 2. Solar MW
df.loc[mask_ap, 'solar_mw_district'] = (
    df.loc[mask_ap, 'solar_pumps_installed'] * PUMP_KW / 1000
)

# 3. Add feeder MW (2020–2025)
df.loc[mask_ap, 'solar_mw_district'] += df.loc[mask_ap].apply(add_feeder_mw, axis=1)

# 4. CO2 reduction
df.loc[mask_ap, 'estimated_co2_reduction_tonnes'] = (
    df.loc[mask_ap, 'solar_mw_district'] *
    1000 * 30 * AVG_KWH_PER_KWP *
    EMISSION_FACTOR / 1000
).round(2)



NameError: name 'ap_pump_targets' is not defined

In [13]:
df.to_csv('ari_energy_filled.csv', index=False)


In [14]:
df.isna().sum()

Unnamed: 0.1                          0
Unnamed: 0                            0
state                                 0
district                              0
year                                  0
month                                 0
solar_mw_district                 23892
wind_mw                           50028
bioenergy_mw                       4356
solar_pumps_installed             10824
estimated_co2_reduction_tonnes    23892
dtype: int64

In [15]:
# Group by state & district and count nulls column-wise
null_summary = (
    df
    .groupby(['state', 'district'], as_index=False)
    .agg(lambda x: x.isna().sum())
)

# Keep only rows where at least one null exists
null_summary_filtered = null_summary[
    null_summary.drop(columns=['state', 'district']).sum(axis=1) > 0
]

null_summary_filtered[45:70]

,state,district,Unnamed: 0.1,Unnamed: 0,year,month,solar_mw_district,wind_mw,bioenergy_mw,solar_pumps_installed,estimated_co2_reduction_tonnes
64,assam,nalbari,0,0,0,0,132,132,0,0,132
65,assam,sivasagar,0,0,0,0,132,132,0,0,132
66,assam,sonitpur,0,0,0,0,132,132,0,0,132
67,assam,tinsukia,0,0,0,0,132,132,0,0,132
68,assam,udalguri,0,0,0,0,132,132,0,0,132
69,bihar,araria,0,0,0,0,132,132,0,0,132
70,bihar,arwal,0,0,0,0,132,132,0,0,132
71,bihar,aurangabad,0,0,0,0,132,132,0,0,132
72,bihar,banka,0,0,0,0,132,132,0,0,132
73,bihar,begusarai,0,0,0,0,132,132,0,0,132


In [16]:
# Assam: PM-KUSUM Solar Pumps (Component-B)
assam_pumps_yearly = {
    2015: 0,
    2016: 50,
    2017: 100,
    2018: 200,
    2019: 300,
    2020: 500,
    2021: 800,
    2022: 1200,
    2023: 2000,
    2024: 3000,
    2025: 4000
}

# Bihar: Solar MW (Standalone + FLS)
bihar_solar_standalone_yearly = {
    2015: 0,
    2016: 10,
    2017: 25,
    2018: 50,
    2019: 80,
    2020: 100,
    2021: 120,
    2022: 140,
    2023: 150,
    2024: 165,
    2025: 180
}

bihar_agri_pv_yearly = {
    2015: 0,
    2016: 20,
    2017: 50,
    2018: 100,
    2019: 150,
    2020: 200,
    2021: 250,
    2022: 280,
    2023: 320,
    2024: 355,
    2025: 390
}

# Bioenergy (MW) approximate
assam_bio_mw = 1.2  # small scale tea/biogas
bihar_bio_mw = 3.4  # rice husk/biomass projects

# Wind MW approximate
assam_wind_mw = 0
bihar_wind_mw = 0.05


In [17]:
def fill_assam_bihar(row):
    state = row['state'].lower()
    year = row['year']
    
    # Assam
    if state == 'assam':
        row['solar_pumps_installed'] = assam_pumps_yearly.get(year, 0)
        row['solar_mw_district'] = round(row['solar_pumps_installed'] * 0.75 / 1000, 3)  # 0.75 kW per pump
        row['bioenergy_mw'] = assam_bio_mw
        row['wind_mw'] = assam_wind_mw
        # CO2 reduction estimate: assume 1 kWh per kWp per day * 30 days * 0.82 kg CO2/kWh
        row['estimated_co2_reduction_tonnes'] = round(row['solar_mw_district'] * 1000 * 4 * 30 * 0.82 / 1000, 2)
        
    # Bihar
    elif state == 'bihar':
        # Sum of standalone + Agri-PV
        solar_total_mw = bihar_solar_standalone_yearly.get(year, 0) + bihar_agri_pv_yearly.get(year, 0)
        row['solar_mw_district'] = solar_total_mw
        row['solar_pumps_installed'] = int(bihar_solar_standalone_yearly.get(year, 0) * 1000 / 0.75)  # convert MW to # pumps approx
        row['bioenergy_mw'] = bihar_bio_mw
        row['wind_mw'] = bihar_wind_mw
        # CO2 reduction
        row['estimated_co2_reduction_tonnes'] = round(row['solar_mw_district'] * 1000 * 4 * 30 * 0.82 / 1000, 2)
    
    return row

df= df.apply(fill_assam_bihar, axis=1)


In [18]:
# Group by state & district and count nulls column-wise
null_summary = (
    df
    .groupby(['state', 'district'], as_index=False)
    .agg(lambda x: x.isna().sum())
)

# Keep only rows where at least one null exists
null_summary_filtered = null_summary[
    null_summary.drop(columns=['state', 'district']).sum(axis=1) > 0
]

null_summary_filtered[45:70]

,state,district,Unnamed: 0.1,Unnamed: 0,year,month,solar_mw_district,wind_mw,bioenergy_mw,solar_pumps_installed,estimated_co2_reduction_tonnes
129,goa,north goa,0,0,0,0,132,132,0,0,132
130,goa,south goa,0,0,0,0,132,132,0,0,132
157,haryana,ambala,0,0,0,0,0,132,0,0,0
158,haryana,bhiwani,0,0,0,0,0,132,0,0,0
159,haryana,faridabad,0,0,0,0,0,132,0,0,0
160,haryana,fatehabad,0,0,0,0,0,132,0,0,0
161,haryana,gurgaon,0,0,0,0,0,132,0,0,0
162,haryana,hisar,0,0,0,0,0,132,0,0,0
163,haryana,jhajjar,0,0,0,0,0,132,0,0,0
164,haryana,jind,0,0,0,0,0,132,0,0,0


In [19]:
df.isna().sum()

Unnamed: 0.1                          0
Unnamed: 0                            0
state                                 0
district                              0
year                                  0
month                                 0
solar_mw_district                 15312
wind_mw                           41448
bioenergy_mw                       4356
solar_pumps_installed             10824
estimated_co2_reduction_tonnes    15312
dtype: int64

In [20]:
df.to_csv('agri_energy_merged.csv', index=False)